In [1]:
#Jai Ganesh Deva

### Convolutional Neural Network Followed by Gru and DNN

Replacing LSTM with a Bidirectional GRU

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize  import word_tokenize
import nltk
from sklearn.decomposition import TruncatedSVD
from sklearn import pipeline,metrics, grid_search
from gensim import models, corpora
from gensim.models import ldamodel
import os
import matplotlib.pyplot as plt
%matplotlib inline
import collections
import io
import keras.backend as K
import csv
import numpy as np
import pandas as pd
import dill as pickle
import gensim
from keras.optimizers import Adam
from gensim.models import word2vec,doc2vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn import metrics

C:\Users\vamsi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\vamsi\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\vamsi\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\vamsi\Anaconda3\lib\site-packages\h5py\__init__.py:36: Fut

In [3]:
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import  Dense, Input,Conv1D,Convolution1D,GRU,Convolution2D ,LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, Masking,MaxPooling1D,GlobalMaxPool1D,Bidirectional,Merge,Flatten,SpatialDropout1D
from keras.models import Model
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import gensim.models.keyedvectors as word2vec
import gc

In [162]:
os.chdir("E:\\Datasets\\Final Project\\Politics Final\\test_balanced")

In [163]:
X_train = pd.read_csv("train_ndf.csv")

In [164]:
X_t

array([[    0,     0,     0, ...,     0,   285,   669],
       [    0,     0,     0, ...,     2,     4,  5379],
       [    0,     0,     0, ...,    18,    84,   748],
       ...,
       [    0,     0,     0, ...,    43,     3, 15446],
       [    0,     0,     0, ...,     7,   639,  4854],
       [    0,     0,     0, ...,     0,     0,  2520]])

In [167]:
X_test = pd.read_csv("test_ndf.csv")

In [168]:
X_test = X_test.drop(['Unnamed: 0', 'Unnamed: 0.1'],axis = 1)

In [183]:
X_train["text_tokens_stop"] = X_train["text_tokens_stop"].str.replace('\d+', '')

In [184]:
X_test["lower_wp"] = X_test["lower_wp"].str.replace('\d+', '')

In [185]:
train_Y = np.array(X_train["sarcastic or not"])

In [186]:
test_Y = np.array(X_test["sarcastic or not"])

In [187]:
list_sentences_train = X_train["lower_wp"]
list_sentences_test = X_test["lower_wp"]

In [188]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
   
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
   
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [189]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [190]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features,char_level=False)

In [191]:
tokenizer.fit_on_texts(list(X_train["lower_wp"]))

In [192]:
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_sentences_test = tokenizer.texts_to_sequences(list_sentences_test)

In [193]:
maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_sentences_test, maxlen=maxlen)



In [228]:
model = Sequential()

In [229]:

model.add(Embedding(len(tokenizer.word_index)+1,embed_size,input_length=maxlen))
model.add(Conv1D(filters = 64,kernel_size=3,padding = "same",activation = "relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(GRU(45, return_sequences=True,name='lstm_layer',dropout=0.4,recurrent_dropout=0.4)))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(1,activation = "sigmoid"))


In [230]:

model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.RMSprop(),
                 metrics=['accuracy',precision,recall, f1])

In [231]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (None, 200, 200)          2640000   
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 200, 64)           38464     
_________________________________________________________________
max_pooling1d_25 (MaxPooling (None, 100, 64)           0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 100, 90)           29700     
_________________________________________________________________
global_max_pooling1d_22 (Glo (None, 90)                0         
_________________________________________________________________
dense_43 (Dense)             (None, 50)                4550      
_________________________________________________________________
dropout_22 (Dropout)         (None, 50)                0         
__________

In [232]:
hist5 = model.fit(X_t,train_Y, batch_size=64, epochs=epochs,validation_split=0.1,callbacks=[earlystop])

Train on 12301 samples, validate on 1367 samples
Epoch 1/4
12301/12301 [==============================] - 63s 5ms/step - loss: 0.6680 - acc: 0.5832 - precision: 0.5939 - recall: 0.6088 - f1: 0.5853 - val_loss: 0.6307 - val_acc: 0.6533 - val_precision: 0.6688 - val_recall: 0.6063 - val_f1: 0.6337
Epoch 2/4
12301/12301 [==============================] - 40s 3ms/step - loss: 0.5811 - acc: 0.7051 - precision: 0.7235 - recall: 0.6749 - f1: 0.6932 - val_loss: 0.6135 - val_acc: 0.6635 - val_precision: 0.6974 - val_recall: 0.5760 - val_f1: 0.6279
Epoch 3/4
12301/12301 [==============================] - 38s 3ms/step - loss: 0.4983 - acc: 0.7655 - precision: 0.7689 - recall: 0.7660 - f1: 0.7627 - val_loss: 0.6286 - val_acc: 0.6620 - val_precision: 0.6476 - val_recall: 0.7123 - val_f1: 0.6773
Epoch 00003: early stopping


In [233]:
pred = model.predict_classes(X_te)

In [234]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

acc = accuracy_score(test_Y,pred)
print(acc)
print(classification_report(test_Y,pred))
print(confusion_matrix(y_true = test_Y,y_pred= pred))


0.6746478873239437
             precision    recall  f1-score   support

          0       0.70      0.62      0.65      1420
          1       0.66      0.73      0.69      1420

avg / total       0.68      0.67      0.67      2840

[[ 876  544]
 [ 380 1040]]


In [147]:
#model.save_weights("cnn+lstm+basic")

In [99]:
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=1, \
                          verbose=1, mode='auto')

In [87]:
hist4 = model.fit(X_t,train_Y, batch_size=batch_size, epochs=epochs,validation_split=0.1,callbacks=[earlystop])

Train on 17080 samples, validate on 1898 samples
Epoch 1/4
17080/17080 [==============================] - 67s 4ms/step - loss: 0.4433 - acc: 0.8006 - precision: 0.8137 - recall: 0.8339 - f1: 0.8186 - val_loss: 0.8945 - val_acc: 0.5337 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: nan
Epoch 2/4
17080/17080 [==============================] - 65s 4ms/step - loss: 0.3110 - acc: 0.8715 - precision: 0.8772 - recall: 0.8973 - f1: 0.8836 - val_loss: 0.9386 - val_acc: 0.5590 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: nan
Epoch 3/4
17080/17080 [==============================] - 65s 4ms/step - loss: 0.1931 - acc: 0.9248 - precision: 0.9287 - recall: 0.9366 - f1: 0.9304 - val_loss: 0.9630 - val_acc: 0.6523 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: nan
Epoch 4/4
17080/17080 [==============================] - 66s 4ms/step - loss: 0.1249 - acc: 0.9529 - precision: 0.9545 - recall: 0.9607 - f1: 0.9564 - val_loss: 1.0926 - val_acc: 0.6581 - val_